Rekall Jupyter Notebook demo
===============

in Reveal.js slides

Original Research by @TekDefense
Notebook, slides assembled by @adricnet

About the OP
-------

We're stepping through the awesome research by Ian Ahl (TekDefense) in the [article](http://www.tekdefense.com/news/2013/12/23/analyzing-darkcomet-in-memory.html) **Analyzing DarkComet in Memory** he wrote after attending and reviewing the Volatility [training](http://www.tekdefense.com/news/2013/12/3/review-malware-and-memory-forensics-with-volatility.html). He pulled some DarkComet samples from AV databases and infected some lab systems to study the malware and developed indicators to help with a case they were working.

About Rekall
------------

* _Rekall_ is a fork of Volatility, from the Tech Preview branch
* Rekall focuses on performance and integration and was originally done to get Vol into [GRR]()
* Rekall embraces newer Python technology including iPython and [notebooks](http://nbviewer.ipython.org/github/ipython/ipython/blob/2.x/examples/Index.ipynb).
* Rekall [Blog](http://rekall-forensic.blogspot.com), [Code](), tutorial [notebook](http://docs.rekall.googlecode.com/git/notebooks/Rekall%20Tutorial.html)




Just a few *iPython* things to get setup to use Rekall:

In [ ]:
from rekall import interactive
interactive.ImportEnvironment()

In [11]:
rekal filename='/Users/adric/Desktop/mess/WIN-TTUMF6EI3O3-20140203-123134.raw'

Initializing Rekall session.
Done!


# Initial analysis

Image information
-----

It's always good to start memory analysis with ***imageinfo***. In Volatility you often have to anyway to find out the profile to use. Rekall doesn't need that since it auto detects profiles handily (detailed at their [blog](http://rekall-forensic.blogspot.com/2014/02/rekall-profiles.html)) but it does help verify that your image is loaded and things are working before you ask harder questions.

In [17]:
imageinfo

Fact                 Value
-------------------- -----
Kernel DTB           0x185000
NT Build             7601.win7sp1_gdr.130828-1532
NT Build Ex          7601.18247.x86fre.win7sp1_gdr.130828-1532
Signed Drivers       True
Time (UTC)           2014-02-03 12:31:36+0000
Time (Local)         2014-02-03 17:31:36+0000
Sec Since Boot       12507.1617736
NtSystemRoot         C:\Windows
**************** Physical Layout ****************
Physical Start Physical End Number of Pages
-------------- ------------ ---------------
0x00001000 0x0009f000 158       
0x00100000 0x3fee0000 261600    
0x3ff00000 0x40000000 256       


Network connections
-----

Usually we can use network connection data to find interesting processes, but not so for this sample as it was prepared without an active malware-related network connection. 

Use ***netscan*** for modern Windows systems and ***connections*** for XP & 2003 (Details in the Volatility command [reference](https://code.google.com/p/volatility/wiki/CommandReference23#Networking)). 

In [21]:
netscan

Offset(P)  Proto    Local Address        Remote Address       State              Pid Owner          Created
---------- -------- -------------------- -------------------- ---------------- ----- -------------- -------
0x3e83a360 TCPv4    0.0.0.0:135          0.0.0.0:0            LISTENING          664 svchost.exe    -      
0x3e83c1d8 TCPv4    0.0.0.0:49152        0.0.0.0:0            LISTENING          388 wininit.exe    -      
0x3e83ca20 TCPv4    0.0.0.0:135          0.0.0.0:0            LISTENING          664 svchost.exe    -      
0x3e83ca20 TCPv6    :::135               :::0                 LISTENING          664 svchost.exe    -      
0x3e840248 TCPv4    0.0.0.0:49152        0.0.0.0:0            LISTENING          388 wininit.exe    -      
0x3e840248 TCPv6    :::49152             :::0                 LISTENING          388 wininit.exe    -      
0x3e861430 TCPv4    0.0.0.0:49153        0.0.0.0:0            LISTENING          752 svchost.exe    -      
0x3e863840 TCPv4    0.0.0.0:

Process table
-----

The ***psxview*** plugin looks through memory in seven (and counting) ways and compares the results to help detect hiding or suspicious processes. 

There are details about the various process plugins in the Volatility command [reference](https://code.google.com/p/volatility/wiki/CommandReference23#Processes_and_DLLs).

In [13]:
psxview

Offset(V)  Name                    PID CSRSS Handles PSScan PsActiveProcessHead PspCidTable Sessions Thrdproc
---------- -------------------- ------ ----- ------- ------ ------------------- ----------- -------- --------
0x85cab690 e                         2 False True    False  False               False       False    False   
0x841389e8 System                    4 False False   True   True                True        False    True    
0x85a68030 taskhost.exe            140 True  True    True   True                True        True     True    
0x84b78020 smss.exe                248 False True    True   True                True        False    True    
0x852e74c8 csrss.exe               336 False True    True   True                True        True     True    
0x849f29c0 dwm.exe                 340 True  True    True   True                True        True     True    
0x855a4388 csrss.exe               380 False True    True   True                True        True     True    
0x855a7bc0

To see the process relationships we can use ***pstree*** just as in Unix.

In [15]:
pstree

Name                                        Pid   PPid   Thds   Hnds Time                    
---------------------------------------- ------ ------ ------ ------ ------------------------
 0x852E74C8:csrss.exe                       336    328      9    394 2014-02-03 09:03:53+0000
 0x855A7BC0:wininit.exe                     388    328      3     81 2014-02-03 09:03:54+0000
. 0x855DC030:services.exe                   484    388      7    199 2014-02-03 09:03:56+0000
.. 0x85A68030:taskhost.exe                  140    484      9    253 2014-02-03 12:13:31+0000
.. 0x856214F8:svchost.exe                   588    484     10    354 2014-02-03 09:04:01+0000
... 0x84AE9190:dllhost.exe                 1920    588      0 ------ 2014-02-03 12:31:38+0000
.. 0x85634030:svchost.exe                   664    484      7    270 2014-02-03 09:04:02+0000
.. 0x85652030:svchost.exe                   752    484     19    476 2014-02-03 09:04:03+0000
... 0x84524030:audiodg.exe                 3936    752      

Anyone in this list look odd? :D

Investigate processes
-----

Let's take a look at the process with the funny name. The ***dlllist*** plugin (three L's or [Tab]) not only lists DLLs loaded by the process but also gives command line information when available.

In [14]:
dlllist pid=1524

************************************************************************
runddl32.exe pid: 1524  
Command line : "C:\Users\TEKDEF~1\AppData\Local\Temp\MSDCSC\runddl32.exe" 
Service Pack 1

   Base       Size    Path
---------- ---------- ----
0x00400000    0xb2000 C:\Users\TEKDEF~1\AppData\Local\Temp\MSDCSC\runddl32.exe
0x76dc0000   0x13c000 C:\Windows\SYSTEM32\ntdll.dll
0x75c20000    0xd4000 C:\Windows\system32\kernel32.dll
0x74f60000    0x4b000 C:\Windows\system32\KERNELBASE.dll
0x75830000    0x8f000 C:\Windows\system32\oleaut32.dll
0x752b0000   0x15c000 C:\Windows\system32\ole32.dll
0x75580000    0xac000 C:\Windows\system32\msvcrt.dll
0x75e20000    0x4e000 C:\Windows\system32\GDI32.dll
0x75b50000    0xc9000 C:\Windows\system32\USER32.dll
0x758c0000     0xa000 C:\Windows\system32\LPK.dll
0x75d80000    0x9d000 C:\Windows\system32\USP10.dll
0x75a90000    0xa2000 C:\Windows\system32\RPCRT4.dll
0x75410000    0xa0000 C:\Windows\system32\advapi32.dll
0x75290000    0x19000 C:\Windows\SYSTEM

Hmm, that doesn't look good. Our suspicious process has a child "notepad" in pstree ... Let's pull it's ***dlllist*** too:

In [16]:
dlllist pid=1896

************************************************************************
notepad.exe pid: 1896  
Command line : notepad
Service Pack 1

   Base       Size    Path
---------- ---------- ----
0x00440000    0x30000 C:\Windows\system32\notepad.exe
0x76dc0000   0x13c000 C:\Windows\SYSTEM32\ntdll.dll
0x75c20000    0xd4000 C:\Windows\system32\kernel32.dll
0x74f60000    0x4b000 C:\Windows\system32\KERNELBASE.dll
0x75410000    0xa0000 C:\Windows\system32\ADVAPI32.dll
0x75580000    0xac000 C:\Windows\system32\msvcrt.dll
0x75290000    0x19000 C:\Windows\SYSTEM32\sechost.dll
0x75a90000    0xa2000 C:\Windows\system32\RPCRT4.dll
0x75e20000    0x4e000 C:\Windows\system32\GDI32.dll
0x75b50000    0xc9000 C:\Windows\system32\USER32.dll
0x758c0000     0xa000 C:\Windows\system32\LPK.dll
0x75d80000    0x9d000 C:\Windows\system32\USP10.dll
0x754c0000    0x7b000 C:\Windows\system32\COMDLG32.dll
0x76f90000    0x57000 C:\Windows\system32\SHLWAPI.dll
0x73d80000   0x19e000 C:\Windows\WinSxS\x86_microsoft.windows

That's a lot of libraries for just notepad. There are many plugins for malware detection and analysis, including ***malfind*** that fuel process investigation further. Of course once you have identified the processes of interest you can extract them to disk for file analysis.

Static analysis : strings
-----

I've already dumped *runddl32.exe_1524.dmp* and *notepad.exe_1896.dmp* for this Rekal notebook demo and strings from both are quite interesting. Following TekDefense's lead and skipping a bit we find the DarkComet configuration variables in the strings for "runddl".

    strings -a runddl32.exe_1524.dmp | grep -A22 DARKCOMET | head -20 > runddl_DC_strings.txt

And here they are:

In [20]:
! cat ~/Desktop/mess/runddl_DC_strings.txt

#BEGIN DARKCOMET DATA --
MUTEX={DC_MUTEX-KHNEW06}
SID={Guest16}
FWB={0}
NETDATA={test213.no-ip.info:1604}
GENCODE={F6FE8i2BxCpu}
INSTALL={1}
COMBOPATH={10}
EDTPATH={MSDCSC\runddl32.exe}
KEYNAME={MicroUpdate}
EDTDATE={16/04/2007}
PERSINST={1}
MELT={0}
CHANGEDATE={1}
DIRATTRIB={6}
FILEATTRIB={6}
SH1={1}
CHIDEF={1}
CHIDED={1}
PERS={1}


As he notes this view into the malware's configuration routines really cracked open the case. He dug in further and did some online research and other testing to discover what the variables affect and the details are in the article.

Additionally, TekDefense found the persistence mechanism in the first place he looked: the HKCU Run key. I'll just include his screenshot for this one so you can see him using ***printkey*** to pull registry key values from memory.


In [25]:
from IPython.display import Image
Image(url='http://www.tekdefense.com/storage/post-images/mp13.png?__SQUARESPACE_CACHEVERSION=1387835722090')

In his research he also tracked down the keylogger files and grabbed the most recent one from memory, which showed him the work he had been doing in his analysis!

Towards indicators
-----

Here's the detailed output from ***malfind*** for "notepad". From the manual:

    The malfind command helps find hidden or injected code/DLLs in user mode memory, based on characteristics such as VAD tag and page permissions. 
    
You can read the details in the command reference [online](https://code.google.com/p/volatility/wiki/CommandReferenceMal23#malfind) and in the *Malware Analyst's Cookbook* (book on [Amazon](http://www.amazon.com/dp/0470613033). It's output quite verbose but as you scroll you'll start to see libraries and syscalls that don't sound right for real notepad before getting down to DarkComet key indicators like the mutex string *DC_MUTEX-KHNEW06*. Using this data @TekData was able to put together some Yara rules for DarkComet detection which he shares in his post.

In [18]:
malfind pid=1896

**************************************************
Process: notepad.exe Pid: 1896 Address: 0x1b0000
Vad Tag: VadS Protection: EXECUTE_READWRITE
Flags: CommitCharge: 1, MemCommit: 1, PrivateMemory: 1, Protection: 6

  0x1b0000 55 dc c6 75 84 cc c6 75 11 ea ba 75 66 c2 c6 75  U..u...u...uf..u 
  0x1b0010 d0 cd c6 75 82 20 c2 75 b0 c2 c6 75 e0 c3 c6 75  ...u...u...u...u 
  0x1b0020 08 f6 de 76 e7 54 c6 75 05 2c c6 75 58 e8 c6 75  ...v.T.u.,.uX..u 
  0x1b0030 3d 61 c7 75 c4 d7 c6 75 00 00 0a 00 00 00 0b 00  =a.u...u........ 

0x001b0000      55                   PUSH EBP                       
0x001b0001      dcc6                 FADD ST6, ST0                  
0x001b0003      7584                 JNZ 0x1aff89                   
0x001b0005      cc                   INT 3                          
0x001b0006      c6                   DB 0xc6                        
0x001b0007      7511                 JNZ 0x1b001a                   
0x001b0009      eaba7566c2c675       JMP FAR 0x75c6:0xc266

And then lastly this "notepad" includes the full path to it's parent process amongst all of the disassembly from *malfind*.

References
------

* Ian Ahl, *Analyzing DarkComet in Memory*, http://www.TekDefence.com

* Volatility Project. *Volatility 2.3 Command Reference*, https://code.google.com/p/volatility/wiki/CommandReference23

* Rekall Project. *Rekall Memory Forensics Tutorial*. Notebook. http://docs.rekall.googlecode.com/git/notebooks/Rekall%20Tutorial.html

* Michael Sikorski and Andrew Honig. *Practical Malware Analysis*. No Starch Press. http://www.nostarch.com/malware

## Questions?

## EOT